# Image validation notebook

## about

### PURPOSE:
This notebook implements a prefilter validation layer for the DermAI skin cancer 
detection system. It ensures that only valid, analyzable dermatological images 
are passed to the main CNN classification model.

### STATEMENT:
Users may upload non-skin images, blurry photos, or low-quality captures that 
cannot be reliably analyzed by the AI model. Processing such images wastes 
computational resources and may produce misleading diagnostic results.

### SOLUTION:
This validation pipeline performs three critical checks before passing images 
to the main diagnostic engine:
  1. Resolution Check - Ensures minimum image dimensions (224x224 pixels)
  2. Blur Detection - Measures image sharpness using Laplacian variance
  3. Skin Detection - Calculates the percentage of skin-tone pixels via HSV analysis

### EXPECTED OUTPUT:
Returns a structured JSON response with:
  - status: "ok" or "error"
  - reason: specific validation failure type (if any)
  - message: user-friendly explanation
  - details: technical metrics (resolution, skin_ratio)

### WORKFLOW:
1. Load image from disk
2. Validate file integrity and format
3. Check minimum resolution requirements
4. Analyze image sharpness (blur detection)
5. Measure skin pixel ratio using color space analysis
6. Return structured validation result

### HOW TO USE:
1. Place test images in the same directory as this notebook
2. Update the 'test_images' list with your image filenames
3. Run all cells sequentially
4. Review JSON output for each image
5. Images with status="ok" are ready for CNN analysis


## 

In [1]:
# import Libraries
import os
import json
import cv2
import numpy as np
from PIL import Image

In [2]:
# config
MIN_WIDTH = 224
MIN_HEIGHT = 224
BLUR_THRESHOLD = 50
SKIN_RATIO_THRESHOLD = 0.20  
TARGET_SIZE = (224, 224)  # Final size for CNN input model

### Build functions

Function: Checks if the image meets the minimum resolution requirements

In [3]:
def check_resolution(img, min_w=MIN_WIDTH, min_h=MIN_HEIGHT):
    h, w = img.shape[:2]
    return w >= min_w and h >= min_h

Function: Detecting blurriness using Laplacian contrast
Idea: Sharp images have high contrast at the edges

In [4]:
def is_blurry(img, threshold=BLUR_THRESHOLD):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    return lap_var < threshold, lap_var

In [5]:
def analyze_skin_texture(img):
    """
    Analyzes image texture to distinguish real skin from non-skin surfaces like walls.
    Skin has: moderate variance, low edge density, natural color distribution.
    
    Returns: (is_skin_like: bool, texture_score: dict)
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 1. Texture variance - skin has moderate roughness
    lap = cv2.Laplacian(gray, cv2.CV_64F)
    texture_var = lap.var()
    
    # 2. Edge density - walls have sharp repetitive edges, skin doesn't
    edges = cv2.Canny(gray, 50, 150)
    edge_density = np.sum(edges > 0) / edges.size
    
    # 3. Standard deviation - measures color variation
    std_dev = np.std(gray)
    
    # Skin characteristics thresholds
    is_skin_like = (
        50 < texture_var < 800 and   # Not too smooth (photo), not too rough (wall)
        edge_density < 0.20 and       # Few sharp edges
        20 < std_dev < 100            # Natural variation, not uniform/pattern
    )
    
    texture_score = {
        "texture_variance": round(texture_var, 2),
        "edge_density": round(edge_density, 4),
        "std_deviation": round(std_dev, 2)
    }
    
    return is_skin_like, texture_score

In [6]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install mediapipe opencv-python

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement mediapipe (from versions: none)
ERROR: No matching distribution found for mediapipe


Function: Create a skin mask using the HSV color space
covering both low and high red values ​​(wrapping around the color circle)

In [8]:
def skin_mask_hsv(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower1 = np.array([0, 20, 70], dtype=np.uint8)
    upper1 = np.array([20, 255, 255], dtype=np.uint8)
    
    lower2 = np.array([160, 20, 70], dtype=np.uint8)
    upper2 = np.array([180, 255, 255], dtype=np.uint8)
    
    mask1 = cv2.inRange(hsv, lower1, upper1)
    mask2 = cv2.inRange(hsv, lower2, upper2)
    
    return cv2.bitwise_or(mask1, mask2)

Function: Creates a skin mask using the YCrCb color space.
More robust in varying lighting conditions.
- Parameters: img: OpenCV image (BGR format)
- Returns: numpy array: Binary mask where skin pixels are white (255)

In [9]:
def skin_mask_ycrcb(img):
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    lower = np.array([0, 133, 77], dtype=np.uint8)
    upper = np.array([255, 173, 127], dtype=np.uint8)
    return cv2.inRange(ycrcb, lower, upper)

Here try to use other tool for skin check

    Enhanced skin detection using both HSV and YCrCb color spaces.
    Combining two methods reduces false positives (e.g., brick walls).
    Returns: float (ratio between 0-1)

In [10]:
# --- Skin Detection using HSV Color Space ---
def skin_ratio_hsv(img):
    
    # HSV color space - good for hue-based detection
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    lower_hsv = np.array([0, 30, 60], dtype=np.uint8)
    upper_hsv = np.array([20, 150, 255], dtype=np.uint8)
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)
    
    # YCrCb color space - more robust in varying lighting
    ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    lower_ycrcb = np.array([0, 133, 77], dtype=np.uint8)
    upper_ycrcb = np.array([255, 173, 127], dtype=np.uint8)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)
    
    # Intersection of both masks - pixel must match BOTH criteria
    combined_mask = cv2.bitwise_and(mask_hsv, mask_ycrcb)
    
    skin_pixels = np.sum(combined_mask > 0)
    total_pixels = combined_mask.size
    ratio = skin_pixels / total_pixels
    
    print(f" Skin ratio: {ratio:.2f}")
    return ratio

Function: Resize image to target dimensions for CNN input.
    Uses INTER_AREA for downscaling (better quality).

In [11]:
def resize_for_model(img, target_size=TARGET_SIZE):
    return cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)

Function analyze_image:
    1. Check file existence
    2. Load and validate image
    3. Check resolution (minimum)
    4. Detect blur
    5. Calculate skin ratio
    6. Resize if valid

In [14]:
def analyze_image(path):
    response = {
        "status": None, 
        "reason": None, 
        "message": None, 
        "details": {},
        "processed_image": None
    }
    
    # File existence check
    if not os.path.exists(path):
        response.update({
            "status": "error", 
            "reason": "file_not_found", 
            "message": "File not found"
        })
        return response
    
    # Load image
    img = cv2.imread(path)
    if img is None:
        response.update({
            "status": "error", 
            "reason": "invalid_image", 
            "message": "Invalid or corrupted image file"
        })
        return response
    
    # Resolution check
    h, w = img.shape[:2]
    response["details"]["original_resolution"] = f"{w}x{h}"
    
    if not check_resolution(img):
        response.update({
            "status": "error", 
            "reason": "low_resolution", 
            "message": f"Image resolution too low (minimum {MIN_WIDTH}x{MIN_HEIGHT})"
        })
        return response
    
    # Blur detection
    blurry, lap_var = is_blurry(img)
    response["details"]["laplacian_variance"] = round(lap_var, 2)
    
    # Skin ratio calculation
    ratio = skin_ratio_hsv(img)
    response["details"]["skin_ratio"] = round(ratio, 4)
    
    # Texture analysis - distinguishes skin from walls/objects
    is_skin_texture, texture_data = analyze_skin_texture(img)
    response["details"]["texture_analysis"] = texture_data
    response["details"]["skin_texture_detected"] = is_skin_texture
    
    # CRITICAL: Check for wall/pattern by edge density
    if texture_data["edge_density"] > 0.25:
        response.update({
            "status": "error", 
            "reason": "not_skin_pattern", 
            "message": "Image contains patterns inconsistent with skin (likely wall/object)"
        })
        return response

    # Check for very rough texture (furniture, wood, etc.)
    if texture_data["texture_variance"] > 1000:
        response.update({
            "status": "error", 
            "reason": "not_skin_texture", 
            "message": "Image texture too rough (likely furniture/wood/object)"
        })
        return response
        
    # Check for very low skin ratio (clearly not skin)
    if ratio < 0.05 and not is_skin_texture:
        response.update({
            "status": "error", 
            "reason": "not_skin", 
            "message": "Insufficient skin area detected"
        })
        return response
    
    # For blurry images, be more strict
    if blurry and ratio < 0.20:
        response.update({
            "status": "error", 
            "reason": "blurry", 
            "message": "Image is too blurry for analysis"
        })
        return response
    
    # Resize for model
    resized_img = resize_for_model(img)
    response["details"]["final_resolution"] = f"{TARGET_SIZE[0]}x{TARGET_SIZE[1]}"
    response["processed_image"] = resized_img
    
    response.update({
        "status": "ok", 
        "message": "Image is valid and ready for analysis"
    })
    return response

In [16]:
# test all images in folder
folder_path = "Test_Images"
print("="*70)
print("IMAGE VALIDATION RESULTS")
print("="*70)

for file in sorted(os.listdir(folder_path)): 
    if file.lower().endswith((".jpg", ".jpeg", ".png")):
        result = analyze_image(os.path.join(folder_path, file))
        result.pop("processed_image", None)
        
        print(f"\n File: {file}")
        print(f"   Status: {' VALID' if result['status'] == 'ok' else ' REJECTED'}")
        
        if result['status'] == 'error':
            print(f"   Reason: {result['reason']}")
        
        print(f"   Message: {result['message']}")
        
        if result['details']:
            print(f"   Details:")
            for key, value in result['details'].items():
                print(f"      • {key}: {value}")
        
        print("-"*70)

IMAGE VALIDATION RESULTS
 Skin ratio: 0.81

 File: HD.jpeg
   Status:  VALID
   Message: Image is valid and ready for analysis
   Details:
      • original_resolution: 3356x4604
      • laplacian_variance: 106.45
      • skin_ratio: 0.8128
      • texture_analysis: {'texture_variance': np.float64(106.45), 'edge_density': np.float64(0.0246), 'std_deviation': np.float64(61.79)}
      • skin_texture_detected: True
      • final_resolution: 224x224
----------------------------------------------------------------------
 Skin ratio: 0.06

 File: ISIC_0000000.jpg
   Status:  VALID
   Message: Image is valid and ready for analysis
   Details:
      • original_resolution: 1022x767
      • laplacian_variance: 58.21
      • skin_ratio: 0.0617
      • texture_analysis: {'texture_variance': np.float64(58.21), 'edge_density': np.float64(0.0048), 'std_deviation': np.float64(66.02)}
      • skin_texture_detected: True
      • final_resolution: 224x224
--------------------------------------------------